In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
n_step = 5
dataset = tf.data.Dataset.from_tensor_slices(tf.range(15))
dataset = dataset.window(n_step, shift=2, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(n_step))
dataset = dataset.shuffle(10).map(lambda window: (window[:-1], window[1:]))
dataset = dataset.batch(3).prefetch(1)

for a in dataset:
    print(a)

(<tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[ 4,  5,  6,  7],
       [ 2,  3,  4,  5],
       [10, 11, 12, 13]])>, <tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[ 5,  6,  7,  8],
       [ 3,  4,  5,  6],
       [11, 12, 13, 14]])>)
(<tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[ 6,  7,  8,  9],
       [ 0,  1,  2,  3],
       [ 8,  9, 10, 11]])>, <tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[ 7,  8,  9, 10],
       [ 1,  2,  3,  4],
       [ 9, 10, 11, 12]])>)


In [3]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [4]:
print(shakespeare_text[:60])

First Citizen:
Before we proceed any further, hear me speak.


In [5]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [6]:
tokenizer.texts_to_sequences(["abcABCdD"])

[[5, 22, 19, 5, 22, 19, 13, 13]]

In [7]:
tokenizer.sequences_to_texts([[5,5,5,5,5,5,13,13]])

['a a a a a a d d']

In [8]:
max_id = len(tokenizer.word_index)
dataset_size = tokenizer.document_count

In [9]:
import numpy as np


[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [10]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [11]:
n_step = 100
window_length = n_step + 1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [12]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [18]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [21]:
dataset = dataset.map(lambda X_batch, y_batch: (tf.one_hot(X_batch, depth=max_id), y_batch))

In [22]:
dataset = dataset.prefetch(1)

In [23]:
for X_batch, y_batch in dataset.take(1):
    print(X_batch.shape, y_batch.shape)

(32, 100, 39) (32, 100)


In [26]:
model = keras.models.Sequential([ 
    keras.layers.GRU(128, return_sequences=True, dropout=0.2, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True, dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax")),
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

model.fit(dataset, epochs=10)

Epoch 1/10
31368/31368 [==============================] - 452s 14ms/step - loss: 1.6216
Epoch 2/10
31368/31368 [==============================] - 456s 15ms/step - loss: 1.5384
Epoch 3/10
31368/31368 [==============================] - 457s 15ms/step - loss: 1.5162
Epoch 4/10
31368/31368 [==============================] - 457s 15ms/step - loss: 1.5041
Epoch 5/10
31368/31368 [==============================] - 457s 15ms/step - loss: 1.4958
Epoch 6/10
31368/31368 [==============================] - 459s 15ms/step - loss: 1.4905
Epoch 7/10
31368/31368 [==============================] - 457s 15ms/step - loss: 1.4863
Epoch 8/10
31368/31368 [==============================] - 457s 15ms/step - loss: 1.4829
Epoch 9/10
31368/31368 [==============================] - 457s 15ms/step - loss: 1.4805
Epoch 10/10
31368/31368 [==============================] - 372s 12ms/step - loss: 1.4782


In [27]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [85]:
X_new = preprocess(["how r yo"])
y_pred = np.argmax(model(X_new), axis=-1)
tokenizer.sequences_to_texts(y_pred + 1)[0]

'e u   i e b o u'

In [86]:
def next_char(text, temperature):
    X_new = preprocess([text])
    y_proba = model(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [107]:
next_char("How are yo", temperature=1)

'u'

In [108]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [149]:
print(complete_text(".", temperature=1))

.

petruchio:

lucentio:
it it she cuest, or it is 


In [152]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_step, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(lambda X_batch, y_batch: (tf.one_hot(X_batch, depth=max_id), y_batch))
dataset = dataset.prefetch(1)

In [164]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_step, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)